In [2]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.utils import shuffle
from keras.utils import to_categorical
import tensorflow as tf
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
DEVICE

'cuda:0'

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


In [6]:
# Parameters
#sequence_length = 50  # Adjust based on your needs
num_features = 9
num_classes = 2
num_epochs = 10

# Data loading and preprocessing function
def load_and_preprocess_data(file_paths, label_paths,folder):
    data = []
    labels = []
    for j in range(40,len(file_paths[folder])):
        #for file_path, label_path in zip(file_paths, label_paths):
        df = pd.read_parquet(file_paths[folder][j])
        #label_df = pd.read_csv(label_path)
        # Assuming binary classification label is in the 'label' column
        data.append(df.values)
    labels.append(label_paths[folder][40:])
    return np.array(data), np.array(labels)

# Generate a list of file and label paths
data_dir = 'D:/Orange/Episafe_local/data/parquet/train/1869'
file_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder).replace(os.path.sep,'/'))]

labels_list =[]
file_paths = []

labels_path = 'D:/Orange/Episafe_local/episafe-data/labels'
for folder in file_folders:
    folder_path = os.path.join(data_dir, folder).replace(os.path.sep,'/')
    folder_files = [os.path.join(folder_path, filename).replace(os.path.sep,'/') for filename in os.listdir(folder_path) if filename.endswith('.parquet')]
    #print(len(folder_files))
    label_file = [os.path.join(labels_path, file).replace(os.path.sep,'/') for file in os.listdir(labels_path) if file.split('_')[1] == folder]
    label_df = pd.read_csv(label_file[0])
    file_paths.append(folder_files)
    file_labels = []
    for i in range(len(os.listdir(folder_path))):
        label_value = label_df['label'][i]
        file_labels.append(label_value)
    labels_list.append(file_labels)
        

# Build the LSTM model
model = tf.keras.Sequential([
    LSTM(units=50, input_shape=(76800, 9)),  # None for variable sequence length
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Training loop
print("************start of training process*************")
for epoch in range(num_epochs):
    #file_paths, label_paths = shuffle(file_paths, file_labels)  # Shuffle data for each epoch
    total_loss = 0
    batch_num = 0
    for folder in range(len(file_paths)):
        batch_num +=1
        batch_x, batch_y = load_and_preprocess_data(file_paths, labels_list,folder)
        #print(batch_y[0])
        batch_loss = model.train_on_batch(batch_x, batch_y[0])
        total_loss += batch_loss  
        model.reset_states()  # Reset states at the end of each batch
        print(f"Epoch {epoch+1}/{num_epochs} - Batch {batch_num} - Loss: {batch_loss:.4f}")
    average_loss = total_loss / len(file_paths)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")
    model.reset_states()  # Reset states at the end of each epoch

# Save the trained model
#model.save('trained_lstm_model.h5')

************start of training process*************


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0
